#
<div style="background-color: gray; padding: 7px;">
    <h2> 1. Criação das Tabelas com o PostgreSQL 
</div>

Para permitir o uso do banco de dados escolhido de forma relacional utilizando SQL, primeiro devemos criá-lo em um Sistema de Gerenciamento de Banco de Dados (SGBD). 

Nesse projeto, utilizamos o `PosgreSQL` e sua interace `pgAdmin`. Esse SGBD é amplamente utilizado por ser de código aberto, confiável e permitir escalabilidade.

Abaixo, disponibilizamos o código necessário para criar as tabelas utilizadas aqui, com as definições de `tipos de dados dos registros` e das `PRIMARY KEYS` e `FOREIGN KEYS`.

``` 
CREATE TABLE categorias 
( 
 id_categoria INTEGER PRIMARY KEY,  -- Identificador único para cada categoria
 nome_categoria VARCHAR(255)  -- Nome da categoria
);

CREATE TABLE clientes 
( 
 id_cliente INTEGER PRIMARY KEY,  -- Identificador único para cada cliente
 nome_cliente VARCHAR(255),  -- Nome do cliente
 idade INTEGER,  -- Idade do cliente
 endereco VARCHAR(255)  -- Endereço do cliente
);

CREATE TABLE fornecedores 
( 
 id_fornecedor INTEGER PRIMARY KEY,  -- Identificador único para cada fornecedor
 nome VARCHAR(255),  -- Nome do fornecedor
 contato VARCHAR(255)  -- Contato do fornecedor (pode ser telefone, email, etc.)
);

CREATE TABLE marcas 
( 
 id_marca INTEGER PRIMARY KEY,  -- Identificador único para cada marca
 nome VARCHAR(255)  -- Nome da marca
);

CREATE TABLE produtos 
( 
 id_produto INTEGER PRIMARY KEY,  -- Identificador único para cada produto
 nome_produto VARCHAR(255),  -- Nome do produto
 preco REAL,  -- Preço do produto
 categoria_id INTEGER,  -- Referência à categoria do produto
 marca_id INTEGER,  -- Referência à marca do produto
 fornecedor_id INTEGER,  -- Referência ao fornecedor do produto
 data_estoque DATE,  -- Data de entrada do produto no estoque
 status VARCHAR(255),  -- Status atual do produto (por exemplo, disponível, esgotado)
 FOREIGN KEY(categoria_id) REFERENCES categorias (id_categoria),  -- Chave estrangeira para a tabela Categorias
 FOREIGN KEY(marca_id) REFERENCES marcas (id_marca),  -- Chave estrangeira para a tabela Marcas
 FOREIGN KEY(fornecedor_id) REFERENCES fornecedores (id_fornecedor)  -- Chave estrangeira para a tabela Fornecedores
);

CREATE TABLE vendas 
( 
 id_venda INTEGER PRIMARY KEY,  -- Identificador único para cada venda
 data_venda DATE,  -- Data em que a venda foi realizada
 total_venda REAL,  -- Valor total da venda
 cliente_id INTEGER,  -- Referência ao cliente que fez a compra
 FOREIGN KEY(cliente_id) REFERENCES clientes (id_cliente)  -- Chave estrangeira para a tabela Clientes
);

CREATE TABLE itens_venda 
( 
 venda_id INTEGER,  -- Referência à venda
 produto_id INTEGER,  -- Referência ao produto vendido
 PRIMARY KEY (venda_id, produto_id),  -- Chave primária composta pelas referências à venda e ao produto
 FOREIGN KEY(venda_id) REFERENCES vendas (id_venda),  -- Chave estrangeira para a tabela Vendas
 FOREIGN KEY(produto_id) REFERENCES produtos (id_produto)  -- Chave estrangeira para a tabela Produtos
);
```

Uma vez definidas no sistema, podemos importar os arquivos csv para cada tabela por meio da interface do pgAdmin.

---

#
<div style="background-color: gray; padding: 7px;">
    <h2> 2. Conexão do Banco com o Jupyter notebook
</div>

In [1]:
# Importando Bibliotecas

import pandas as pd 
from sqlalchemy import create_engine

In [2]:
# Configuração de Conexão com o Banco de Dados PostgreSQL

USER = "postgres"   # Usuário cadastrado
SENHA = "postgres"  # Senha
HOST = "localhost"  # Host do servidor PostgreSQL
DATABASE = "black"  # Nome do banco de dados

engine = create_engine(f"postgresql+psycopg2://{USER}:{SENHA}@{HOST}/{DATABASE}")

### Ferramentas:
- Git 
- PandaSQL 
- PostgreSQL 
- pgAdmin4

### Referências:
- https://pandas.pydata.org
- https://www.postgresql.org
- https://pypi.org/project/pandasql/
- https://pypi.org/project/psycopg2/
- https://learnsql.com/blog/how-to-import-csv-to-postgresql/

---

#
<div style="background-color: gray; padding: 10px;">
    <h2> 3. Exploração Inicial dos Dados
</div>

- Tabela ``categorias``;

In [18]:
query = """
SELECT * FROM categorias
"""

df = pd.read_sql_query(query, con = engine)
print(f'Shape da tabela categorias {df.shape}')
df

Shape da tabela categorias (5, 2)


id_categoria nome_categoria
0             1    Eletrônicos
1             2      Vestuário
2             3      Alimentos
3             4         Livros
4             5       Esportes

Essa é uma tabela dimensão que armazena atributos das categorias de produtos desse negócio. O `id_categoria`, que é a `PRIMARY-KEY`, é o identificador único do tipo ``Int`` para cada categoria distinta presente, cujos nomes são representados por `nome_categoria` do tipo ``Object``.

Com o `df.shape` sabemos que essa tabela tem 5 linhas e 2 colunas.

Observamos que há somente 5 categorias: Eletrônicos, Vestuário, Alimentos, Livros e Esportes.

Utilizando o comando `DataFrame.info()` da biblioteca `pandas`, verificamos os tipos de dados de cada coluna da tabela convertida em dataframe.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_categoria    5 non-null      int64 
 1   nome_categoria  5 non-null      object
dtypes: int64(1), object(1)
memory usage: 212.0+ bytes


Também podemos usar o comando `DataFrame.isnull().sum()` para verificar se temos dados faltantes. 

Nessa tabela, não há dados faltantes.

In [20]:
df.isnull().sum()

id_categoria      0
nome_categoria    0
dtype: int64

---

- Tabela ``clientes``;

In [21]:
query = """
SELECT * FROM clientes
"""

df = pd.read_sql_query(query, con = engine)
print(f'Shape da tabela clientes {df.shape}')
df.head()

Shape da tabela clientes (10000, 4)


id_cliente nome_cliente  idade endereco
0           1        Carla     50       PR
1           2       Felipe     36       RJ
2           3        Elena     42       DF
3           4        Alice     29       PA
4           5        Alice     44       MA

A tabela clientes contém os atributos de nome, idade e endereço de cada cliente desse negócio, sendo `id_cliente` a `PRIMARY KEY`.

Aqui, temos 10.000 linhas e 4 colunas, cujos tipos de dados podemos observar a seguir:

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_cliente    10000 non-null  int64 
 1   nome_cliente  10000 non-null  object
 2   idade         10000 non-null  int64 
 3   endereco      10000 non-null  object
dtypes: int64(2), object(2)
memory usage: 312.6+ KB


Nesta tabela não temos dados faltantes.

In [23]:
df.isnull().sum()

id_cliente      0
nome_cliente    0
idade           0
endereco        0
dtype: int64

---

- Tabela ``fornecedores``;

In [24]:
query = """
SELECT * FROM fornecedores
"""

df = pd.read_sql_query(query, con = engine)
print(f'Shape da tabela fornecedores {df.shape}')
df.head()

Shape da tabela fornecedores (10, 3)


id_fornecedor                 nome                          contato
0              1          AstroSupply          contato@astrosupply.com
1              2  HorizonDistributors  contato@horizondistributors.com
2              3        EchoLogistics        contato@echologistics.com
3              4    CosmicConnections    contato@cosmicconnections.com
4              5           TerraTrade           contato@terratrade.com

Essa tabela representa os fornecedores, identificados pela `PRIMARY KEY` em `id_fornecedor` e com informações de nome e contato para cada um deles. 

Pelo tamanho dessa, vemos que há 10 fornecedores presentes.

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_fornecedor  10 non-null     int64 
 1   nome           10 non-null     object
 2   contato        10 non-null     object
dtypes: int64(1), object(2)
memory usage: 372.0+ bytes


Também não possuimos dados faltantes nessa tabela.

In [26]:
df.isnull().sum()

id_fornecedor    0
nome             0
contato          0
dtype: int64

---

- Tabela ``marcas``;

In [27]:
query = """
SELECT * FROM marcas
"""

df = pd.read_sql_query(query, con = engine)
print(f'Shape da tabela marcas {df.shape}')
df

Shape da tabela marcas (10, 2)


id_marca          nome
0         1      BluePeak
1         2    ZenithWave
2         3    SolarFlare
3         4     EchoBloom
4         5  CrystalCrest
5         6    NovaSphere
6         7    GreenPulse
7         8  SilverStream
8         9    AmberField
9        10  InfinityAura

Essa tabela representa as marcas dos produtos, identificados pela `PRIMARY KEY` em `id_marca` com informações de nome.

Pelo tamanho, vemos que há 10 marcas de produtos distintas.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_marca  10 non-null     int64 
 1   nome      10 non-null     object
dtypes: int64(1), object(1)
memory usage: 292.0+ bytes


Também não possuimos dados faltantes nessa tabela.

In [8]:
df.isnull().sum()

id_marca    0
nome        0
dtype: int64

---

- Tabela ``produtos``;

In [9]:
query = """
SELECT * FROM produtos
"""

df = pd.read_sql_query(query, con = engine)
df.tail(5)

id_produto     nome_produto    preco  categoria_id  marca_id  \
9995        9996  Livro de Ficção  1567.81             5         9   
9996        9997          Celular  1690.56             5        10   
9997        9998          Celular  1278.29             2         9   
9998        9999  Livro de Ficção   521.94             1         2   
9999       10000  Livro de Ficção  1075.08             5         7   

      fornecedor_id data_estoque   status  
9995              1   2021-12-25  Vendido  
9996              9   2020-10-19  Vendido  
9997              5   2022-04-07  Vendido  
9998              7   2021-04-23  Estoque  
9999              6   2023-08-15  Vendido

Essa tabela representa os produtos, identificados pela `PRIMARY KEY` em `id_produto` e informações de nome em ``nome_produto``

 O ``preço`` do produto é representado por uma coluna do tipo ``Float``. 

Além disso, temos ``categoria_id``, ``categoria_id`` e ``categoria_id``, que são ``FOREIGN-KEYS`` referentes a outras tabelas. 

A coluna ``data_estoque`` indica a data em que o produto foi adicionado ao estoque e a coluna ``status`` indica se um produto foi vendido ou ainda está em estoque, ambas do tipo ``Object``

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_produto     10000 non-null  int64  
 1   nome_produto   10000 non-null  object 
 2   preco          10000 non-null  float64
 3   categoria_id   10000 non-null  int64  
 4   marca_id       10000 non-null  int64  
 5   fornecedor_id  10000 non-null  int64  
 6   data_estoque   10000 non-null  object 
 7   status         10000 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 625.1+ KB


Não possuimos dados faltantes na tabela de produtos. 

In [11]:
df.isnull().sum()

id_produto       0
nome_produto     0
preco            0
categoria_id     0
marca_id         0
fornecedor_id    0
data_estoque     0
status           0
dtype: int64

---

- Tabela ``vendas``;

In [12]:
query = """
SELECT * FROM vendas
"""

df = pd.read_sql_query(query, con = engine)
df.head()

id_venda  data_venda  total_venda  cliente_id
0         1  2021-05-01       167.21        8112
1         2  2023-01-12      1315.33          76
2         3  2023-06-09      3044.65        1285
3         4  2021-01-31      4576.87        3897
4         5  2022-02-09      3429.25        2749

Aqui temos ``id_venda`` que é a ``PRIMARY-KEY`` desta tabela, junto com a ``data_venda`` que é data da compra.

Também temos ``total_venda`` que representa o valor da venda que também o somatório dos preços dos produtos.

Por fim, o ``cliente_id`` que é uma ``FOREIGN-KEY`` da tabela clientes.

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_venda     50000 non-null  int64  
 1   data_venda   50000 non-null  object 
 2   total_venda  50000 non-null  float64
 3   cliente_id   50000 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 1.5+ MB


Não possuimos dados faltantes na tabela de vendas. 

In [14]:
df.isnull().sum()

id_venda       0
data_venda     0
total_venda    0
cliente_id     0
dtype: int64

---

- Tabela ``itens_venda``;

In [15]:
query = """
SELECT * FROM itens_venda
"""

df = pd.read_sql_query(query, con = engine)
df.head()

venda_id  produto_id
0         1        1510
1         2        1762
2         3        6889
3         3        5477
4         3        2181

A tabela ``itens_venda`` é uma tabela fato que apenas junta a ``venda_id`` e ``produto_id`` relacionando quais foram os produtos adquiridos em cada venda.

---

#
<div style="background-color: gray; padding: 10px;">
    <h2> 4. Análise Exploratória
</div>

##
<div style="background-color: gray; padding: 0px;">
    <h2> 4.1 Performance das Vendas
</div>

- Quais marcas e fornecedores tiveram melhor e pior desempenho em termos de volume de vendas.
- Quais categorias tiveram melhor e pior desempenho em termos de volume de vendas.
- Comparar desempenho de fornecedores durante o periodo
- Plotar gráficos fornecedores x periodo
- Porcentagem de categorias, fornecedores e marcas vendidas

##
<div style="background-color: gray; padding: 0px;">
    <h2> 4.2 Sazonalidade das Vendas
</div>

- Relacionar quantidade de vendas por Ano/Mes
- Identificar sazonalidades
- Comparar os anos 
- Prever como sera a black friday de 2023